In [9]:
import tinyarray as ta
import numpy as np
import matplotlib.pyplot as plt
import importlib
import json
import kwant

import codes.trijunction as trijunction
import codes.optimization as optimization
import codes.parameters as parameters
from codes.tools import hamiltonian
from codes.utils import eigsh, svd_transformation, dict_update, dep_acc_index
from codes.utils import order_wavefunctions

In [2]:
from scipy.optimize import minimize, minimize_scalar

In [3]:
with open('/home/tinkerer/trijunction-design/codes/config.json', 'r') as f:
    config = json.load(f)

change_config = [
    {"device": {"thickness": {"dielectric": 1.0}}},
    {"gate": {"channel_width": 13.0, 'angle': np.pi/6, 'gap': 2}},
]

for local_config in change_config:
    config = dict_update(config, local_config)

In [4]:
system = trijunction.Trijunction(config)

In [5]:
system.make_system()

Finding closed voronoi cells
Done selecting closed voronoi cells
Calculating points distance
Done
Calculating ridges hypersurfaces
Done
Finding  hypervolume for closed voronoi cells
Done finding hypervolume for closed voronoi cells
Done calculating surface
Done calculating distance
Done calculating capacitance matrix


100%|██████████| 13/13 [00:31<00:00,  2.41s/it]


In [6]:
pair = 'left-right'
voltages = parameters.pair_voltages()

In [7]:
zero_potential = dict(
    zip(
        ta.array(system.site_coords[:, [0, 1]] - system.offset),
        np.zeros(len(system.site_coords)),
    )
)

In [10]:
kwant_args = list(system.optimiser_arguments().values())
kwant_args[-1] = kwant_args[-1][order_wavefunctions(pair)]

In [11]:
x = [-3.0e-3, -3.0e-3, -7.0e-3, 3e-3]
voltages = parameters.voltage_dict(x)

In [12]:
params = parameters.junction_parameters()
params.update(voltages)
params.update(potential=zero_potential)
params.update(voltages)

In [13]:
opt_args = tuple([pair, params, kwant_args])

In [14]:
%%time

sol = minimize_scalar(
    optimization.loss,
    0.2, 
    args=opt_args, 
    bounds=(0, 2), 
    method="bounded"
)

CPU times: user 49.9 s, sys: 11.9 s, total: 1min 1s
Wall time: 1min 1s


In [15]:
optimal_phase = parameters.phase_pairs(opt_args[0], 
                                       sol.x * np.pi)

In [16]:
optimal_phase

{'phi1': 4.303469493233111, 'phi2': 0}

In [18]:
# loss to optimize voltages
params.update(optimal_phase)
opt_args = tuple([pair, params, kwant_args])
optimization.loss(x, *opt_args)

3.6413198571384764

In [ ]:
sol1 = minimize(
            optimization.loss,
            x,
            args=opt_args,
            method="trust-constr",
            options={
                "verbose": 2,
                "initial_tr_radius": 1e-3,
                # "gtol": 1e-1,
            },
        )

| niter |f evals|CG iter|  obj func   |tr radius |   opt    |  c viol  |
|-------|-------|-------|-------------|----------|----------|----------|
|   1   |   5   |   0   | +3.6413e+00 | 1.00e-03 | 2.41e+03 | 0.00e+00 |
|   2   |  10   |   1   | -2.1138e+00 | 7.00e-03 | 6.44e+03 | 0.00e+00 |
|   3   |  15   |   2   | -5.1649e+00 | 7.00e-03 | 4.05e+01 | 0.00e+00 |
|   4   |  20   |   3   | -5.1652e+00 | 7.00e-03 | 4.03e+01 | 0.00e+00 |
|   5   |  25   |   5   | -5.1652e+00 | 1.61e-03 | 4.03e+01 | 0.00e+00 |
|   6   |  30   |   6   | -5.1652e+00 | 8.07e-04 | 4.03e+01 | 0.00e+00 |
|   7   |  35   |   7   | -5.1652e+00 | 2.99e-04 | 4.03e+01 | 0.00e+00 |
|   8   |  40   |   8   | -5.1889e+00 | 2.09e-03 | 1.59e+02 | 0.00e+00 |
|   9   |  45   |  11   | -5.1889e+00 | 2.53e-04 | 1.59e+02 | 0.00e+00 |
|  10   |  50   |  13   | -5.2199e+00 | 1.77e-03 | 8.09e+02 | 0.00e+00 |
|  11   |  55   |  15   | -5.2199e+00 | 8.87e-04 | 8.09e+02 | 0.00e+00 |
|  12   |  60   |  18   | -5.2199e+00 | 2.96e-04 | 

In [93]:
params.update(parameters.voltage_dict(sol1.x))
opt_args = tuple([pair, params, kwant_args])

In [59]:
optimiser_args = system.optimiser_arguments()

numerical_hamiltonian = hamiltonian(system.trijunction, 
                                    optimiser_args['linear_terms'], 
                                    optimiser_args['kwant_params_fn'], 
                                    **params)

In [61]:
energies, wave_functions = eigsh(
                numerical_hamiltonian.tocsc(),
                6,
                sigma=0,
                return_eigenvectors=True,
)

In [75]:
transformed_hamiltonian = svd_transformation(
    energies, wave_functions, reference_wave_functions
)

desired = np.abs(transformed_hamiltonian[0, 1])
undesired = np.linalg.norm(transformed_hamiltonian[2:])

In [77]:
desired

2.430561344761288e-05